In [1]:
import subprocess, os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("token")
repo = "/kaggle/working/RSNABreastCancer2023"

In [2]:
%%capture
if not os.path.isdir(repo):
    subprocess.run(["git", "clone", f"https://{token}@github.com/isaiahchua/RSNABreastCancer2023.git"])

Cloning into 'RSNABreastCancer2023'...


In [3]:
os.chdir(repo)
! git pull

Already up to date.


In [4]:
! pip install --no-index --find-links=/kaggle/input/monai-whl monai
! pip install --no-index --find-links=/kaggle/input/torchio torchio
! pip install --no-index --find-links=/kaggle/input/addict addict

Looking in links: /kaggle/input/monai-whl
Processing /kaggle/input/monai-whl/monai-1.1.0-202212191849-py3-none-any.whl
Looking in links: /kaggle/input/torchio
Processing /kaggle/input/torchio/torchio-0.18.86-py2.py3-none-any.whl
Processing /kaggle/input/torchio/Deprecated-1.2.13-py2.py3-none-any.whl
Processing /kaggle/input/torchio/shellingham-1.5.0.post1-py2.py3-none-any.whl
Looking in links: /kaggle/input/addict
Processing /kaggle/input/addict/addict-2.4.0-py3-none-any.whl


In [5]:
import os, sys, platform
from os.path import join, dirname, basename, abspath
import time
import numpy as np
import pandas as pd
import pydicom
import SimpleITK as sitk
import cv2
import torch
import tensorflow
import matplotlib.pyplot as plt
import yaml
import h5py
import json
# import addict
from addict import Dict
# import monai
import monai
# import torchio
import torchio as tio
# import custom classes
import importlib
import submission
importlib.reload(submission)
from submission import Submission

In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

print(f"python version: {sys.version}")
print(f"os type: {os.name}")
print(f"os: {platform.system()}")
print(f"os release: {platform.release()}")

train_count = 0
test_count  = 0
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if "dcm" in filename:
            if 'test_images' in dirname:
                test_count += 1
            else:
                train_count += 1
        else:
            print(os.path.join(dirname, filename))

print("total train:", train_count)
print("total test :", test_count)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
os type: posix
os: Linux
os release: 5.15.65+
/kaggle/input/monai-whl/monai-1.1.0-202212191849-py3-none-any.whl
/kaggle/input/addict/addict-2.4.0-py3-none-any.whl
/kaggle/input/rsna-breast-cancer-detection/sample_submission.csv
/kaggle/input/rsna-breast-cancer-detection/train.csv
/kaggle/input/rsna-breast-cancer-detection/test.csv
/kaggle/input/torchio/torchio-0.18.86-py2.py3-none-any.whl
/kaggle/input/torchio/Deprecated-1.2.13-py2.py3-none-any.whl
/kaggle/input/torchio/shellingham-1.5.0.post1-py2.py3-none-any.whl
/kaggle/input/rsna-breast-cancer-model-weights/20230122/densenet_final_01.pth
total train: 54706
total test : 4


## Preprocessing Test Images

In [7]:
! python preprocessing.py -c /kaggle/working/RSNABreastCancer2023/config/test_config.yaml

Metadata file created in /kaggle/working/preprocessed/metasub.json.
/kaggle/working/preprocessed/mammosub224.h5 created.
Timesheet created in /kaggle/working/preprocessed/sub_timesheet.json.


In [8]:
! python splitdata.py -c ./config/test_config.yaml

## Load and run model

In [9]:
cfile = "/kaggle/working/RSNABreastCancer2023/config/test_config.yaml"
if torch.cuda.is_available():
    torch.cuda.empty_cache()
cfgs = Dict(yaml.load(open(abspath(cfile), "r"), Loader=yaml.Loader))
submit = Submission(cfgs)
start = time.time()
submit.Run()
submit.ExportSubmissionCSV()
end = time.time()
print("\n")
start_s = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(start))
end_s = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(end))
time_taken = end - start
if time_taken > 3600.:
    divisor = 3600.
    suffix = "hr"
else:
    divisor = 60.
    suffix = "min"
print(f"Start Time: {start_s}, End Time: {end_s}, Total Time Taken: {(time_taken)/divisor:.3f} {suffix}")



Start Time: 2023-01-31 05:06:33, End Time: 2023-01-31 05:06:34, Total Time Taken: 0.004 min
